### Download

In [ ]:
### set years interval for weather data
yearBegin = 2021 # year begin
yearEnd = 2022 # year end

In [ ]:
### load weather zone and weather station information
zoneList = pd.read_excel("Data/ERCOT weather stations.xlsx")

In [ ]:
### Create a new directory for weather year if it does not exist 
for i in range(yearBegin,yearEnd+1):
    path = "Data/%s" % (i)
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
### download weather data for each zone, each year
for i in range(yearBegin,yearEnd+1):
    path = "Data/%s" % (i)
    for j in range(len(zoneList)):
        url = "https://www.ncei.noaa.gov/data/global-hourly/access/%s/%s.csv" % (i, zoneList["STATION_ID"][j])
        wget.download(url,out = path)

### NOAA temperature data processing


In [ ]:
### set years interval for weather data
yearBegin = 2021 # year begin
yearEnd = 2022 # year end

In [ ]:
### load weather zone and weather station information
zoneList = pd.read_excel("Data/ERCOT weather stations.xlsx")

In [ ]:
### Create a new directory for weather year if it does not exist 
if not os.path.exists("Data/Temp"):
    os.makedirs("Data/Temp")

In [ ]:
for i in range(len(zoneList)):
    df = pd.DataFrame()
    for j in range(yearBegin,yearEnd+1):
        tempRaw = pd.read_csv("Data/%s/%s.csv" % (j,zoneList["STATION_ID"][i]), usecols=["DATE", "REPORT_TYPE", "TMP"])
        df = pd.concat([df, tempRaw])
    df = df.loc[df["REPORT_TYPE"]=="FM-15"]
    df["DATE"] = pd.to_datetime(df["DATE"])
    df["DATE"] = df["DATE"].round('H')
    df = df.drop_duplicates(subset=['DATE'], keep='first')
    df = df.set_index("DATE")
    df["TMP"] = df["TMP"].str.slice(stop=5)
    df["TMP"] = df["TMP"].astype(int)/10
    ### clear temperature outlier
    df = df[(np.abs(stats.zscore(df["TMP"])) < 3)]
    ### fill missing data
    idx = pd.date_range('01-01-2021 01:00', '12-31-2022 23:55',freq="H")
    df = df.reindex(idx)
    ### potentially calculate missing data number here
    df = df.fillna(method = "backfill")
#     temps = df['2021-06-27':'2022-06-26'] ### hard code here
    df.to_csv("Data/Temp/%s.csv" % (zoneList["Weather Zone"][i]))